https://stepik.org/lesson/309343/step/7

Пусть в этот раз на входе батч из одного трехслойного (RGB) изображения размером 3 * 3.

Пусть ядро имеет 2 фильтра шириной и высотой 2 пикселя.

Тогда выход должен иметь размерность 1 * 2 * 2 * 2.

<img src='https://ucarecdn.com/ddc6ccbe-2aef-4c7b-a47b-a15e67d3f6ec/' width=800>

### Рекомендуем проверить на листочке, что результат произведения таких матриц дает тот же результат, что и свертка

<img src='https://ucarecdn.com/afbc3c3c-a347-4248-b0de-cd614e637fc3/' width=800>

In [ ]:
#@title Работает при разных batch_size, не работает при изменении input_height, input_width или stride
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

""" ЗДЕСЬ ЗАДАЁМ РАЗМЕРЫ БАТЧА, КАРТИНОК И STRIDE: """
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        converted_kernel = self.kernel.reshape(-1, self.kernel.shape[1] * self.kernel.shape[2] * self.kernel.shape[3])
        return converted_kernel


    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width): # здесь тоже исходное ядро, это self.kernel
        # Реализуйте преобразование входа
        converted_input = torch.zeros((self.kernel.shape[1] * self.kernel.shape[2] * self.kernel.shape[3], torch_input.shape[0]))
        # Перебор входного тензора и извлечение нужной ячейки
        for b in range(torch_input.shape[0]):  # Перебор по батчам (столбцы выхода)
            count = 0                          # счётчик строки, в которую будем писать, обнуляется при новом батче
            for i in range(self.kernel.shape[1]):  # Перебор по каналам (для RGB их три)
                for j in range(self.kernel_size):  # Перебор по высоте ядра
                    for k in range(self.kernel_size):  # Перебор по ширине ядра

                        # Извлекаем ячейку из входного тензора
                        region = torch_input[b, i, j, k]
                        # Записывааем её на нужное место
                        converted_input[count, b] = region # (по одной ячейке, в нужный столбец)
                        count += 1 # переход к следующей строке

        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        # print('Результат перемножения развёрнутых матриц:', converted_kernel @ converted_input)
        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(1,0).view(torch_input.shape[0],
                                                     self.out_channels,
                                                     output_height,
                                                     output_width)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dMatrixV2))

Результат перемножения развёрнутых матриц: tensor([[ 5252., 12596., 19940., 27284., 34628.]])
True
